In [ ]:
# prompt: kết nối với gg drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
! pip install joblib underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import joblib
import re
from underthesea import word_tokenize
import numpy as np

In [ ]:
def load_and_preprocess_data(file_path, stop_words_path):
    data = pd.read_csv(file_path)
    print(data.isnull().sum())
    if 'Unnamed: 3' in data.columns:
        data = data.drop(columns=['Unnamed: 3'])
    print(data["label"].unique())
    stop_words_df = pd.read_csv(stop_words_path, header=None)
    stop_words = set(stop_words_df[0].values)
    def process_sentences(sentence):
        sentence = re.sub(r'[^\w\d]', ' ', sentence)
        sentence_tokenizer = word_tokenize(sentence)
        sentence_tokenizer = [
            word.lower().replace(" ", "_")
            for word in sentence_tokenizer
            if word not in stop_words and len(word) > 1
        ]
        return " ".join(sentence_tokenizer)

    data['comment'] = data['comment'].apply(process_sentences)
    data = data.drop(columns=['rate'], axis=1)
    train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
    return train_data, test_data
train_data, test_data = load_and_preprocess_data("/content/drive/MyDrive/NLP/Data/data - data.csv", "/content/drive/MyDrive/NLP/Data/vietnamese-stopwords-dash.txt")

comment           0
label             0
rate              0
Unnamed: 3    31437
dtype: int64
['POS' 'NEG' 'NEU']


In [ ]:
train_data

,comment,label
28079,chất_lượng sản_phẩm tuyệt_vời shop phục_vụ rất...,POS
9072,có đáp_bụng chun bầu bí_mặc quần thoải_mái lắm...,POS
24976,sẽ tiếp_tục ủng_hộ shop,POS
9577,chất_lượng sản_phẩm tuyệt_vờ đóng_gói sản_phẩm...,POS
23682,tất_cute lắm đi thoải_mái nha shop,POS
...,...,...
29802,chất_lượng sản_phẩm tuyệt_vời đóng_gói sản_phẩ...,POS
5390,,NEU
860,sp sale đẹp chất_lượng,POS
15795,va bao da y_dă se_u ng hô_shop ng ma_u tiê_p đ...,POS


In [ ]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(train_data["label"].to_list())
y_test = encoder.transform(test_data["label"].to_list())
joblib.dump(encoder, './data/label_encoder.pkl')
encoder = joblib.load('./data/label_encoder.pkl')
y_train_decoded = encoder.inverse_transform(y_train)

# DL

## pho_bert

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer, AdamW, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
class Sentiment_Dataset(Dataset):
  def __init__(self, tokenizer, comments, labels, max_length=258):
    self.tokenizer = tokenizer
    self.comments = comments
    self.labels = labels
    self.max_length = max_length

  def __len__(self):
    return len(self.comments)

  def __getitem__(self, idx):
    comment = self.comments[idx]
    label = self.labels[idx]
    encoding = self.tokenizer.encode_plus(
        comment,
        add_special_tokens=True,
        max_length=self.max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids = encoding['input_ids'].flatten()
    attention_mask = encoding['attention_mask'].flatten()

    return {
        'comment': comment,
        'input_ids':input_ids,
        'attention_masks': attention_mask,
        'label': label
    }

In [ ]:
class BertClassifier(nn.Module):
    def __init__(self):
        super(BertClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("vinai/phobert-base-v2")
        self.drop = nn.Dropout(p=0.3)
        self.fc = nn.Sequential(
            nn.Linear(in_features=self.bert.config.hidden_size, out_features=32),
            nn.ReLU(),
            nn.LayerNorm(32),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=32, out_features=8),
            nn.ReLU(),
            nn.LayerNorm(8),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=8, out_features=3),
            nn.Softmax(1)
        )
    def forward(self, input_ids, attention_mask):
        last_hidden_state, output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False
        )
        x = self.drop(output)
        x = self.fc(x)
        return x

In [ ]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(train_data["label"].to_list())
y_test = encoder.transform(test_data["label"].to_list())

In [ ]:
phobert = "vinai/phobert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(phobert)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [ ]:
train_dataset = Sentiment_Dataset(tokenizer, train_data['comment'].to_list(), y_train)
test_dataset = Sentiment_Dataset(tokenizer, test_data['comment'].to_list(), y_test)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

In [ ]:
def train(train_dataloader, model, loss_fn, optimizer, epochs=50, val_dataloader=None, validation=False, checkpoint=False, checkpoint_path="", save_folder="model", save_metrics="val_accuracy"):
  min_val_loss = 9999
  max_val_acc = 0
  last_epoch = 0
  model = model.to(device)
  loss_fn = loss_fn.to(device)
  if checkpoint:
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    last_epoch = checkpoint['epoch']
    val_loss = checkpoint['loss']
    min_val_loss = val_loss.item()
  num_epochs = last_epoch+epochs
  history = {"loss": [],
    "accuracy": [],
    "val_loss": [],
    "val_accuracy": [],
  }
  for epoch in range(last_epoch, num_epochs):
    model.train()
    process_bar = tqdm(train_dataloader)
    total_loss = 0
    total_acc = 0
    count = 0
    for step, data in enumerate(process_bar):
      input_ids = data['input_ids'].to(device)
      attention_mask = data['attention_masks'].to(device)

      labels = data['label'].to(device)
      output = model(
        input_ids=input_ids, attention_mask=attention_mask,
      )
      train_loss = loss_fn(output, labels)
      train_loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      pred = torch.argmax(output, 1)-1
      labels = labels - 1
      train_acc = (labels == pred).sum()/len(labels)
      del input_ids, attention_mask, labels
      process_bar.set_description("Epoch: {}/{}, Loss: {:0.8f}, Accuracy: {:0.2f}".format(epoch+1, num_epochs, train_loss.item(), train_acc))

      total_loss += train_loss.item()
      total_acc += train_acc.cpu()
      count += 1
    history["loss"].append(total_loss/count)
    history["accuracy"].append(total_acc/count)
    if validation:
      all_losses = []
      all_predictions = []
      all_labels = []
      model.eval()
      with torch.no_grad():
        for iter, data in enumerate(val_dataloader):
          input_ids = data['input_ids'].to(device)
          attention_mask = data['attention_masks'].to(device)
          labels = data['label'].to(device)

          optimizer.zero_grad()
          output = model(
              input_ids=input_ids,
              attention_mask=attention_mask,
          )
          val_loss = loss_fn(output, labels)
          pred = torch.argmax(output, 1)-1
          labels = labels - 1
          all_labels.extend(labels.tolist())
          all_predictions.extend(pred)
          all_losses.append(val_loss.item())
          del input_ids, attention_mask, labels
        acc = (torch.Tensor(all_labels) == torch.Tensor(all_predictions)).sum()/len(all_labels)
        total_val_loss = sum(all_losses)/len(all_losses)
        if save_metrics == 'val_accuracy' and acc > max_val_acc:
          max_val_acc = acc
          torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': val_loss,
              'accuracy': max_val_acc,
          }, "/content/drive/MyDrive/NLP/Model/best_checkpoint.pt")
        elif total_val_loss < min_val_loss:
          min_val_loss = total_val_loss
          torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_loss,
            'accuracy': max_val_acc,
        }, "/content/drive/MyDrive/NLP/Model/best_checkpoint.pt")
        print("Epoch: {}/{}, loss: {:0.8f}, accuracy: {:0.8f}, val_loss: {:0.8f}, val_accuracy: {:0.8f}".format(epoch+1, num_epochs, total_loss/count, total_acc/count, total_val_loss, acc))
        history["val_loss"].append(total_val_loss)
        history["val_accuracy"].append(acc.cpu())
  return history

In [ ]:
def eval(model, val_dataloader):
  all_predictions = []
  all_labels = []
  model = model.to(device)
  model.eval()

  with torch.no_grad():
    for iter, data in enumerate(val_dataloader):
      input_ids = data['input_ids'].to(device)
      attention_mask = data['attention_masks'].to(device)
      labels = data['label'].to(device)
      output = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
      )
      pred = torch.argmax(output, 1)-1
      labels = labels - 1
      all_labels.extend(labels.tolist())
      all_predictions.extend(pred)
      del input_ids, attention_mask, labels
    all_predictions = [x.item() for x in all_predictions]
    print(classification_report(all_labels, all_predictions))
    return all_labels, all_predictions

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
loss_fn = nn.CrossEntropyLoss()
classifier = BertClassifier()

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = torch.optim.AdamW([
    {"params": classifier.bert.parameters()},
    {"params": classifier.fc.parameters()}
], lr=2e-5, weight_decay = 2e-5)

In [ ]:
classifier

BertClassifier(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Lay

In [ ]:
history = train(
    train_loader,
    model = classifier,
    loss_fn=loss_fn,
    optimizer=optimizer,
    epochs=50,
    val_dataloader=test_loader,
    validation=True,
    save_folder="/content/drive/MyDrive/NLP/Model/best.pt"
    )

  0%|          | 0/394 [00:00<?, ?it/s]

Epoch: 1/50, loss: 0.97356477, accuracy: 0.66184169, val_loss: 0.92030467, val_accuracy: 0.73124605


  0%|          | 0/394 [00:00<?, ?it/s]

Epoch: 2/50, loss: 0.92787321, accuracy: 0.73524743, val_loss: 0.89445001, val_accuracy: 0.75349653


  0%|          | 0/394 [00:00<?, ?it/s]

...

  0%|          | 0/394 [00:00<?, ?it/s]

Epoch: 10/50, loss: 0.83799585, accuracy: 0.77014595, val_loss: 0.82639735, val_accuracy: 0.75937700


  0%|          | 0/394 [00:00<?, ?it/s]

Epoch: 11/50, loss: 0.83083122, accuracy: 0.77093911, val_loss: 0.81921308, val_accuracy: 0.76605213


  0%|          | 0/394 [00:00<?, ?it/s]

## Pho bert wonrax/phobert-base-vietnamese-sentiment

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
class BertClassifier(nn.Module):
    def __init__(self):
        super(BertClassifier, self).__init__()
        self.bert = AutoModelForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")
        # self.drop = nn.Dropout(p=0.3)
        # self.fc = nn.Sequential(
        #     nn.Linear(in_features=self.bert.config.hidden_size, out_features=32),
        #     nn.ReLU(),
        #     nn.LayerNorm(32),
        #     nn.Dropout(p=0.3),
        #     nn.Linear(in_features=32, out_features=8),
        #     nn.ReLU(),
        #     nn.LayerNorm(8),
        #     nn.Dropout(p=0.3),
        #     nn.Linear(in_features=8, out_features=3),
        #     nn.Softmax(1)
        # )
    def forward(self, input_ids, attention_mask):
        output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False
        )
        # x = self.drop(output)
        # x = self.fc(x)
        return output[0]

In [ ]:
classifier = BertClassifier()

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

In [ ]:
classifier

BertClassifier(
  (bert): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(64001, 768, padding_idx=1)
        (position_embeddings): Embedding(258, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dens

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW([
    {"params": classifier.bert.parameters()}
], lr=2e-5, weight_decay = 2e-5)

In [ ]:
history = train(
    train_loader,
    model = classifier,
    loss_fn=loss_fn,
    optimizer=optimizer,
    epochs=50,
    val_dataloader=test_loader,
    validation=True,
    save_folder="/content/drive/MyDrive/NLP/Model/best_checkpoint.pt"
    )

  0%|          | 0/394 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

Epoch: 1/50, loss: 0.59623041, accuracy: 0.76705265, val_loss: 0.58147098, val_accuracy: 0.76748252


  0%|          | 0/394 [00:00<?, ?it/s]

Epoch: 2/50, loss: 0.51467071, accuracy: 0.79441625, val_loss: 0.57203804, val_accuracy: 0.77193260


...

Epoch: 8/50, loss: 0.32138672, accuracy: 0.88309008, val_loss: 0.70116847, val_accuracy: 0.75667512


  0%|          | 0/394 [00:00<?, ?it/s]

Epoch: 9/50, loss: 0.29551118, accuracy: 0.89296478, val_loss: 0.73794110, val_accuracy: 0.76509851


  0%|          | 0/394 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
tokenizer.save_pretrained("/content/drive/MyDrive/NLP/Model/tokeinizer_phobert_base")

('/content/drive/MyDrive/NLP/Model/tokeinizer_phobert_base/tokenizer_config.json',
 '/content/drive/MyDrive/NLP/Model/tokeinizer_phobert_base/special_tokens_map.json',
 '/content/drive/MyDrive/NLP/Model/tokeinizer_phobert_base/vocab.txt',
 '/content/drive/MyDrive/NLP/Model/tokeinizer_phobert_base/bpe.codes',
 '/content/drive/MyDrive/NLP/Model/tokeinizer_phobert_base/added_tokens.json')